In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:

# Get API keys from environment variables
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
GROK_API_KEY = os.getenv("GROK_API_KEY")
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT", "financer")

LANGSMITH_TRACING=True

In [ ]:
llm= ChatOpenAI(model_name="gpt-4o",)
print(llm.invoke("What is the capital of France?"))

In [76]:
import yfinance as yf
from datetime import datetime
import feedparser
def get_yfinance_news(ticker, max_items=5):
    try:
        news = yf.Ticker(ticker).news
        print(len(news))
        if not news:
            return "No news available for this ticker."
        result = []
        for item in news:
            print(len(item),item)
            # if result and len(result) >= max_items:
            #     break
            # # print(item)
            # published = item.get('providerPublishTime')
            # if published:
            #     published = datetime.utcfromtimestamp(published).strftime('%Y-%m-%d %H:%M:%S')
            # else:
            #     published = ''
            
            result.append({
                'title': item['title'],
                'link': item['link'],
                # 'published': published
            })
            print(len(result))
        return result
    except Exception:
        return []




In [77]:
print(get_yfinance_news("TSLA", 5))

10
2 {'id': '4bdef547-e9b8-30a8-bdeb-5b040e294730', 'content': {'id': '4bdef547-e9b8-30a8-bdeb-5b040e294730', 'contentType': 'STORY', 'title': 'Electric Vehicles Today - European Battery Testing Market Accelerates with Regulatory Boost', 'description': '', 'summary': 'The European market for electric vehicle (EV) battery testing, inspection, and certification is set to experience significant growth, projected to escalate from $410.9 million in 2024 to $2.21 billion by 2034. This expansion is driven by stringent EU regulations, increased adoption of EVs, and advances in testing technologies, which ensure battery safety and compliance. The market growth is further supported by EU incentives and sustainability goals, with innovative technologies like AI...', 'pubDate': '2025-06-23T10:39:34Z', 'displayTime': '2025-06-23T10:39:34Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': {'originalUrl': 'https://media.zenfs.com/en/simply_wall_st__316/8a9a68e8370d8514f2f282d

In [73]:
import yfinance as yf
res=list(yf.Ticker("TSLA").news)
print(res)

[{'id': '10087f91-8b65-35af-88cc-d491f891e6f8', 'content': {'id': '10087f91-8b65-35af-88cc-d491f891e6f8', 'contentType': 'STORY', 'title': 'Musk’s robotaxis caught speeding as they hit roads for the first time', 'description': '', 'summary': 'Elon Musk’s Tesla robotaxis have been caught speeding and driving into oncoming traffic as the billionaire’s driverless ride-hailing service hits roads for the first time.', 'pubDate': '2025-06-23T09:11:50Z', 'displayTime': '2025-06-23T09:11:50Z', 'isHosted': True, 'bypassModal': False, 'previewUrl': None, 'thumbnail': None, 'provider': {'displayName': 'The Telegraph', 'url': 'http://www.telegraph.co.uk/'}, 'canonicalUrl': {'url': 'https://uk.finance.yahoo.com/news/musk-robotaxis-caught-speeding-hit-091150644.html', 'site': 'finance', 'region': 'GB', 'lang': 'en-GB'}, 'clickThroughUrl': {'url': 'https://finance.yahoo.com/news/musk-robotaxis-caught-speeding-hit-091150644.html', 'site': 'finance', 'region': 'US', 'lang': 'en-US'}, 'metadata': {'edit

In [ ]:
def get_google_news(query, max_items=5):
    url = f"https://news.google.com/rss/search?q={query.replace(' ', '+')}"
    feed = feedparser.parse(url)
    return [
        {
            'title': entry.title,
            'link': entry.link,
            'published': entry.published
        } for entry in feed.entries[:max_items]
    ]

In [ ]:

def build_news_sentiment_prompt(company_news, world_news):
    prompt = (
        "You are a professional financial analyst. When answering, ALWAYS consider both company-specific news and broader world news (such as wars, geopolitics, economic events) "
        "as these can significantly impact the market and your analysis. "
        "Tell the user that both sources are being used in your reasoning.\n\n"
        "Below are recent company-specific news headlines:\n"
    )
    for i, item in enumerate(company_news, 1):
        prompt += f"{i}. {item['title']} ({item['published']})\n"
    prompt += "\nRecent broader/world news headlines:\n"
    for i, item in enumerate(world_news, 1):
        prompt += f"{i}. {item['title']} ({item['published']})\n"
    prompt += (
        "\nFirst, summarize the sentiment of each news set; then combine their insights into your final answer or suggestion. "
        "Explicitly state to the user that both company and world news have been considered.\n"
        "Format:\n"
        "1. Company news sentiment summary\n"
        "2. World news sentiment summary\n"
        "3. Final answer, clearly noting both sources were used.\n"
    )
    return prompt
